In [ ]:
import metrics
import pandas as pd

TEST_FILE = "data/test-train_non_mc_qa_test_mc.tsv"
PREDICTION_FILE = "data/predictions/train_non_mc_qa_test_mc_predictions"

# Read test file
df = pd.read_csv(TEST_FILE, sep="\t", header=None)
n_col = df.shape[1]
all_targets = df[range(4, n_col)].apply(
    lambda row: row[~row.isnull()].tolist(), axis=1)
df = df[range(4)]
df.columns = ["task_name", "task_prefix", "input", "target"]
df["all_targets"] = all_targets

# Read prediction file
with open(PREDICTION_FILE) as fin:
    predictions = fin.readlines()
predictions = [x.replace("\n", "") for x in predictions]
df["prediction"] = predictions

df = df.groupby(["task_name", "task_prefix"]).agg({
    'all_targets': lambda x: list(x),
    'prediction': lambda x: list(x)}).reset_index()

def compute_f1(row):
    return metrics.evaluate(row.prediction, row.all_targets, "QA-F1")
    #return metrics.get_f1_over_list(row.prediction, row.all_targets)

df["f1_score"] = df.apply(compute_f1, axis=1)
df.groupby(["task_name"]).agg({'f1_score': ["mean", "var"]})